In [1]:
import numpy as np
import scipy.stats as sps
from tqdm import tqdm
import pandas as pd
from statsmodels.sandbox.stats.multicomp import multipletests

# Комбинирование критериев

$(X_1, ..., X_n), (Y_1, ..., Y_m)$ --- выборки

Являются ли они нормальными или нет, однородными или нет?

$\mathsf{H}_1$: выборка $(X_1, ..., X_n)$ нормальна

$\mathsf{H}_2$: выборка $(Y_1, ..., Y_m)$ нормальна

$\mathsf{H}_3$: выборки однородны


In [2]:
def test(x, y, alpha=0.05):
    """
    Реализация комбинации критериев с условием FWER <= alpha. 
    Возвращает три булевских значения. Если True, то соответствующая гипотеза отклоняется.
    """
    
    result = np.zeros(3, dtype=bool)
    
    # Первая процедура
    rejected = multipletests([# критерий Шапиро-Уилка для выборки X
                              sps.shapiro(x)[1],
                              # критерий Шапиро-Уилка для выборки Y
                              sps.shapiro(y)[1],
                              # критерий Стьюдента для проверки однородности
                              sps.ttest_ind(x, y)[1]], 
                             method='holm',
                             alpha=alpha)[0]
    
    # Результат проверки нормальности сразу пишем в ответ
    result[:2] = rejected[:2]
    
    # Если нормальность отвергается хотя бы для одной выборки,
    # то для проверки однородности реализуем вторую процедуру,
    # в которой в данном случае только критерий Уилкоксона-Манна-Уитни.
    # Результат применения этой процедуры пишем в ответ.
    if rejected[:2].any():
        result[2] = sps.mannwhitneyu(x, y, alternative='two-sided')[1] < alpha
        
    # Если нормальность не отвергается, то в ответ пишем результат Стьюдента
    else:
        result[2] = rejected[2]
        
    return result


def result_table(rejected):
    """ Составляет красивую таблицу """
    
    # Трюк с категориальными нужен, чтобы явно указать множество значений.
    # Если этого не сделать, то при отсутствии какого-то значения все полетит.
    table = pd.crosstab(index=pd.Categorical(rejected[:, :2].any(axis=1), 
                                             categories=[False, True]), 
                        columns=pd.Categorical(rejected[:, 2],  
                                               categories=[False, True]), 
                        normalize=True)
    
    table.index = ['Нормальность не отвергается', 
                   'Нормальность отвергается хотя бы для одной']
    table.columns = ['Однородность не отвергается', 
                     'Однородность отвергается']
    
    return table

Параметры для всех запусков

In [3]:
size = 100  # размер выборки
n_iter = 200000  # количество итераций в одном эксперименте
alpha = 0.05  # уровень значимости

### Эксперимент 1

Все гипотезы верны.
Ошибка первого рода происходит, если хотя бы одна гипотеза отвергается.
Из таблицы видно, что бОльшая часть ошибок приходится на отвержение нормальности.
Это объясняется тем, что для этого используется два критерия, причем доля соответствующих ошибок примерно в 2 раза больше доле случаев отвержения однородности.

In [4]:
rejected = np.zeros((n_iter, 3),  dtype=bool)
x = sps.norm.rvs(size=(n_iter, size))
y = sps.norm.rvs(size=(n_iter, size))

for i in tqdm(range(n_iter)):
    rejected[i] = test(x[i], y[i])
    
n_errors = rejected.any(axis=1).sum()
print('FWER = {:.4f} +/- {:.4f}'.format(n_errors / n_iter, 2 * np.sqrt(alpha / n_iter)))
result_table(rejected)

100%|██████████| 200000/200000 [1:14:33<00:00, 44.70it/s]


FWER = 0.0495 +/- 0.0010


,Однородность не отвергается,Однородность отвергается
Нормальность не отвергается,0.950535,0.016020
Нормальность отвергается хотя бы для одной,0.031070,0.002375


### Эксперимент 2

Выборки нормальные, но имеют разное среднее.
Ошибка первого рода происходит, если отвергается нормальность хотя бы одной из выборок.
Во всех случаях комбинированный критерий верно отверг однородность.

In [5]:
rejected = np.zeros((n_iter, 3),  dtype=bool)
x = sps.norm.rvs(size=(n_iter, size))
y = sps.norm(loc=1).rvs(size=(n_iter, size))

for i in tqdm(range(n_iter)):
    rejected[i] = test(x[i], y[i])
    
n_errors = rejected[:, :2].any(axis=1).sum()
print('FWER = {:.4f} +/- {:.4f}'.format(n_errors / n_iter, 2 * np.sqrt(alpha / n_iter)))
result_table(rejected)

100%|██████████| 200000/200000 [1:15:42<00:00, 44.03it/s]


FWER = 0.0490 +/- 0.0010


,Однородность не отвергается,Однородность отвергается
Нормальность не отвергается,0.0,0.951015
Нормальность отвергается хотя бы для одной,0.0,0.048985


### Эксперимент 3

Выборки не являются нормальными, но при этом имеют распределение Стьюдента с 10 степенями свободы, которое достаточно похоже на нормальное.
Ошибка первого рода происходит только при отвержении однородности, что происходит только примерно в $\alpha / 2$ случаев.
Почти в 3/4 случаев нормальность не отвергается, т.е. предположения критерия Стьюдента, вообще говоря, не выполнены.
Тем не менее, он хорошо работает, поскольку выборки в целом обладают некоторыми свойствами нормальности.

In [6]:
rejected = np.zeros((n_iter, 3),  dtype=bool)
x = sps.t(df=10).rvs(size=(n_iter, size))
y = sps.t(df=10).rvs(size=(n_iter, size))

for i in tqdm(range(n_iter)):
    rejected[i] = test(x[i], y[i])
    
n_errors = rejected[:, 2].sum()
print('FWER = {:.4f} +/- {:.4f}'.format(n_errors / n_iter, 2 * np.sqrt(alpha / n_iter)))
result_table(rejected)

100%|██████████| 200000/200000 [1:21:09<00:00, 41.07it/s]


FWER = 0.0258 +/- 0.0010


,Однородность не отвергается,Однородность отвергается
Нормальность не отвергается,0.71739,0.01102
Нормальность отвергается хотя бы для одной,0.25682,0.01477


### Эксперимент 4

Аналогично предыдущему случаю, но используется распределение Стьюдента с 5 степенями свободы, которое уже менее похоже на нормальное.
Это приводит к тому, что почти в 3/4 случаев нормальность все же отвергается.
Реальное значение FWER выше, чем в предыдущем случает, но уровня $\alpha$ не достигает.

In [7]:
rejected = np.zeros((n_iter, 3),  dtype=bool)
x = sps.t(df=5).rvs(size=(n_iter, size))
y = sps.t(df=5).rvs(size=(n_iter, size))

for i in tqdm(range(n_iter)):
    rejected[i] = test(x[i], y[i])
    
n_errors = rejected[:, 2].sum()
print('FWER = {:.4f} +/- {:.4f}'.format(n_errors / n_iter, 2 * np.sqrt(alpha / n_iter)))
result_table(rejected)

100%|██████████| 200000/200000 [1:23:41<00:00, 39.83it/s]


FWER = 0.0407 +/- 0.0010


,Однородность не отвергается,Однородность отвергается
Нормальность не отвергается,0.288485,0.003995
Нормальность отвергается хотя бы для одной,0.670800,0.036720


### Эксперимент 5

Выборки по-прежнему однородны, но при этом имеют экспоненциальное распределение, которое совсем не похоже на нормальное.
В каждом случае нормальность была отвергнута хотя бы для одной выборки (а может всегда для обоих выборок), поэтому однородность всегда проверялась во второй процедуре.

In [8]:
rejected = np.zeros((n_iter, 3),  dtype=bool)
x = sps.expon.rvs(size=(n_iter, size))
y = sps.expon.rvs(size=(n_iter, size))

for i in tqdm(range(n_iter)):
    rejected[i] = test(x[i], y[i])
    
n_errors = rejected[:, 2].sum()
print('FWER = {:.4f} +/- {:.4f}'.format(n_errors / n_iter, 2 * np.sqrt(alpha / n_iter)))
result_table(rejected)

100%|██████████| 200000/200000 [1:22:52<00:00, 40.22it/s]


FWER = 0.0497 +/- 0.0010


,Однородность не отвергается,Однородность отвергается
Нормальность не отвергается,0.00000,0.00000
Нормальность отвергается хотя бы для одной,0.95031,0.04969


---------

Прикладная статистика и анализ данных, 2019

Никита Волков

https://mipt-stats.gitlab.io/